In [1]:
"""
Load a BehaviourStore setup json and interactively edit it.
"""
from behaviourstoreeditor.behaviourstorefoldereditor import *


In [2]:
storeeditor = BehaviourStoreFolderEditor()
storeeditor.get_widgets()